In [1]:
using Revise,DelimitedFiles,Random,StaticArrays
using ParallelTemperingMonteCarlo,StructArrays
using BenchmarkTools


Ensemble, potential, params etc

In [16]:
n_atoms = 38

ti = 300.
tf = 1300.
n_traj = 28

temp = TempGrid{n_traj}(ti,tf) 

mc_cycles = 10000
mc_sample = 1

displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)

evtohartree = 0.0367493
nmtobohr = 18.8973

#parameters taken from L Vocadlo etal J Chem Phys V120N6 2004
n = 8.482
m = 4.692
ϵ = evtohartree*0.0370
a = 0.25*nmtobohr
C = 27.561

pot = EmbeddedAtomPotential(n,m,ϵ,C,a)
ensemble = NVT(n_atoms)
move_strat = MoveStrategy(ensemble)



pos_cu38 = [[2.3603476948363165, 2.3603476948363165, 0.0],
[6.994369407022418, 2.33998871029911, 0.0],
[2.33998871029911, 6.994369407022418, 0.0],
[-2.3603476948363165, 2.3603476948363165, 0.0],
[-6.994369407022418, 2.33998871029911, 0.0],
[-2.33998871029911, 6.994369407022418, 0.0],
[-2.3603476948363165, -2.3603476948363165, 0.0],
[-6.994369407022418, -2.33998871029911, 0.0],
[-2.33998871029911, -6.994369407022418, 0.0],
[2.3603476948363165, -2.3603476948363165, 0.0],
[6.994369407022418, -2.33998871029911, 0.0],
[2.33998871029911, -6.994369407022418, 0.0],
[0.0, 0.0, 3.3380357219419614],
[4.84532317769689, 0.0, 3.4261608756649893],
[-4.84532317769689, 0.0, 3.4261608756649893],
[0.0, 4.84532317769689, 3.4261608756649893],
[0.0, -4.84532317769689, 3.4261608756649893],
[4.667179058660764, 4.667179058660764, 3.2911441531516483],
[-4.667179058660764, 4.667179058660764, 3.2911441531516483],
[-4.667179058660764, -4.667179058660764, 3.2911441531516483],
[4.667179058660764, -4.667179058660764, 3.2911441531516483],
[0.0, 0.0, -3.3380357219419614],
[4.84532317769689, 0.0, -3.4261608756649893],
[-4.84532317769689, 0.0, -3.4261608756649893],
[0.0, 4.84532317769689, -3.4261608756649893],
[0.0, -4.84532317769689, -3.4261608756649893],
[4.667179058660764, 4.667179058660764, -3.2911441531516483],
[-4.667179058660764, 4.667179058660764, -3.2911441531516483],
[-4.667179058660764, -4.667179058660764, -3.2911441531516483],
[4.667179058660764, -4.667179058660764, -3.2911441531516483], 
[2.327190348361654, 2.327190348361654, 6.600387922922003],
[-2.327190348361654, 2.327190348361654, 6.600387922922003],
[-2.327190348361654, -2.327190348361654, 6.600387922922003],
[2.327190348361654, -2.327190348361654, 6.600387922922003],
[2.327190348361654, 2.327190348361654, -6.600387922922003],
[-2.327190348361654, 2.327190348361654, -6.600387922922003],
[-2.327190348361654, -2.327190348361654, -6.600387922922003],
[2.327190348361654, -2.327190348361654, -6.600387922922003]]

n_bin = 100
AtoBohr = 1.8897259886

bc_cu38 = SphericalBC(radius=14*AtoBohr) 
start_config = Config(pos_cu38, bc_cu38)


Config{38, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[2.3603476948363165, 2.3603476948363165, 0.0], [6.994369407022418, 2.33998871029911, 0.0], [2.33998871029911, 6.994369407022418, 0.0], [-2.3603476948363165, 2.3603476948363165, 0.0], [-6.994369407022418, 2.33998871029911, 0.0], [-2.33998871029911, 6.994369407022418, 0.0], [-2.3603476948363165, -2.3603476948363165, 0.0], [-6.994369407022418, -2.33998871029911, 0.0], [-2.33998871029911, -6.994369407022418, 0.0], [2.3603476948363165, -2.3603476948363165, 0.0]  …  [-4.667179058660764, -4.667179058660764, -3.2911441531516483], [4.667179058660764, -4.667179058660764, -3.2911441531516483], [2.327190348361654, 2.327190348361654, 6.600387922922003], [-2.327190348361654, 2.327190348361654, 6.600387922922003], [-2.327190348361654, -2.327190348361654, 6.600387922922003], [2.327190348361654, -2.327190348361654, 6.600387922922003], [2.327190348361654, 2.327190348361654, -6.600387922922003], [-2.327190348361654, 2.327190348361654, -6.60038

RunnerPotential

In [14]:
script_folder = @__DIR__ # folder where this script is located
data_path = joinpath(script_folder, "data") # path to data files, so "./data/"

X = [ 2              0.001   0.000  11.338
 #10              0.001   0.000  11.338
 2              0.020   0.000  11.338
 #10              0.020   0.000  11.338
 2              0.035   0.000  11.338
 #10              0.035   0.000  11.338
 2              0.100   0.000  11.338
 #10              0.100   0.000  11.338
 2              0.400   0.000  11.338
 #10              0.400   0.000  11.338
 ]

radsymmvec = []


#--------------------------------------------#
#--------Vector of angular symm values-------#
#--------------------------------------------#
V = [[0.0001,1,1,11.338],[0.0001,-1,2,11.338],[0.003,-1,1,11.338],[0.003,-1,2,11.338],[0.008,-1,1,11.338],[0.008,-1,2,11.338],[0.008,1,2,11.338],[0.015,1,1,11.338],[0.015,-1,2,11.338],[0.015,-1,4,11.338],[0.015,-1,16,11.338],[0.025,-1,1,11.338],[0.025,1,1,11.338],[0.025,1,2,11.338],[0.025,-1,4,11.338],[0.025,-1,16,11.338],[0.025,1,16,11.338],[0.045,1,1,11.338],[0.045,-1,2,11.338],[0.045,-1,4,11.338],[0.045,1,4,11.338],[0.045,1,16,11.338],[0.08,1,1,11.338],[0.08,-1,2,11.338],[0.08,-1,4,11.338],[0.08,1,4,11.338]]

T = [111,110,100]

angularsymmvec = []
#-------------------------------------------#
#-----------Including scaling data----------#
#-------------------------------------------#
file = open(joinpath(data_path,"scaling.data")) # full path "./data/scaling.data"
scalingvalues = readdlm(file)
close(file)
G_value_vec = []
for row in eachrow(scalingvalues[1:88,:])
    max_min = [row[4],row[3]]
    push!(G_value_vec,max_min)
end


for symmindex in eachindex(eachrow(X))
    row = X[symmindex,:]
    radsymm = RadialType2{Float64}(row[2],row[4],Int(row[1]),G_value_vec[symmindex])
    push!(radsymmvec,radsymm)
end


let n_index = 10

for element in V
    #for types in T

        n_index += 1

        symmfunc = AngularType3{Float64}(element[1],element[2],element[3],11.338,2,G_value_vec[n_index])

        push!(angularsymmvec,symmfunc)
    #end
end
end
#---------------------------------------------------#
#------concatenating radial and angular values------#
#---------------------------------------------------#

totalsymmvec = vcat(radsymmvec,angularsymmvec)


#--------------------------------------------------#
#-----------Initialising the nnp weights-----------#
#--------------------------------------------------#
num_nodes::Vector{Int32} = [88, 20, 20, 1]
activation_functions::Vector{Int32} = [1, 2, 2, 1]
file = open(joinpath(data_path, "weights.029.data"), "r+") # "./data/weights.029.data"
weights=readdlm(file)
close(file)
weights = vec(weights)
nnp = NeuralNetworkPotential(num_nodes,activation_functions,weights)

#runnerpotential = RuNNerPotential(nnp,radsymmvec,angularsymmvec)
struct RuNNerPotential2Atom{Nrad,Nang,N1,N2} <: AbstractMachineLearningPotential
    nnp1::NeuralNetworkPotential
    nnp2::NeuralNetworkPotential
    radsymfunctions::StructVector{RadialType2{Float64}}
    angsymfunctions::StructVector{AngularType3{Float64}}
    r_cut::Float64 
end
function RuNNerPotential2Atom(nnp1,nnp2,radsymvec,angsymvec,n1,n2)
    r_cut = radsymvec[1].r_cut
    nrad = length(radsymvec)
    nang = length(angsymvec)
    radvec=StructVector([rsymm for rsymm in radsymvec])
    angvec = StructVector([asymm for asymm in angsymvec])

    return RuNNerPotential2Atom{nrad,nang,n1,n2}(nnp1,nnp2,radvec,angvec,r_cut)
end
file = open(joinpath(data_path, "weights.029.data"), "r+") # "./data/weights.029.data"
weights=readdlm(file)
close(file)
weights = vec(weights)
nnpcu = NeuralNetworkPotential(num_nodes,activation_functions,weights)

file2=open(joinpath(data_path, "weights.030.data"),"r+") #./data/weights.030.data
weights2=readdlm(file2)
close(file2)
weights2 = vec(weights2)
nnpzn= NeuralNetworkPotential(num_nodes,activation_functions,weights2)


runnerpotential = RuNNerPotential2Atom(nnpcu,nnpzn,radsymmvec,angularsymmvec,30,8)

RuNNerPotential2Atom{5, 26, 30, 8}(NeuralNetworkPotential(4, 2221, Int32[88, 20, 20, 1], Int32[1, 2, 2, 1], [-1.0768943782, 0.3563458393, -1.3084861447, 0.1127640916, -0.5646591931, -0.4969830793, 1.423770514, -0.2005662393, -0.8957859374, 0.6076110858  …  0.1939524848, -0.0342903191, -0.0231042009, 0.0718481968, -0.0828320122, -0.0728501885, 0.0306628826, 0.0486053813, -0.0867018862, 0.0441651841]), NeuralNetworkPotential(4, 2221, Int32[88, 20, 20, 1], Int32[1, 2, 2, 1], [-0.2990596476, 0.50210302, -0.0769338744, -1.1006272074, 0.2955859359, 1.7429579447, -1.6013368538, 0.2350663304, 0.5352038521, 0.0051334333  …  -0.07023029, -0.0274223406, 0.0379881304, 0.1032826216, -0.0671415566, 0.0873441204, 0.0024981831, -0.1005392202, 0.011513595, -0.0270711056]), RadialType2{Float64}[RadialType2{Float64}(0.001, 11.338, 2, -0.029892621990594427, 0.06634475788973929), RadialType2{Float64}(0.02, 11.338, 2, -0.0, 0.09828103647521905), RadialType2{Float64}(0.035, 11.338, 2, -0.013430592462258787, 

In [4]:
testvector = rand(2,8)
test2vector = testvector[2:-1:1,:]

2×8 Matrix{Float64}:
 0.338326  0.128342  0.244797  0.15654   …  0.670691  0.368588  0.481922
 0.691093  0.175848  0.446453  0.915256     0.311685  0.871261  0.680434

In [24]:
function calc_symm_vals!(position,dist2_mat,f_mat,g_vec,n1,n2,symm_func::RadialType2)
    N = n1+n2
    g_norm,η = symm_func.G_norm,symm_func.eta
    for atomindex in 1:n1
        for index2 in atomindex+1:n1

            g_val =  calc_one_symm_val(dist2_mat[atomindex,index2],f_mat[atomindex,index2],g_norm,η)

            g_vec[1,atomindex] += g_val 
            g_vec[1,index2] += g_val
        end
    end

    for atomindex in n1+1:N
        for index2 in atomindex+1:N

            g_val =  calc_one_symm_val(dist2_mat[atomindex,index2],f_mat[atomindex,index2],g_norm,η)

            g_vec[1,atomindex] += g_val 
            g_vec[1,index2] += g_val
        end
    end

    for atomindex in 1:n1
        for index2 in n1+1:N

            g_val =  calc_one_symm_val(dist2_mat[atomindex,index2],f_mat[atomindex,index2],g_norm,η)

            g_vec[2,atomindex] += g_val 
            g_vec[2,index2] += g_val
        end
    end

    return g_vec
end

calc_symm_vals! (generic function with 2 methods)

In [62]:
function calc_symm_vals!(positions,dist2_mat,f_mat,g_vec,n1,n2,symm_func::AngularType3)
    N= n1+n2
    η,λ,ζ,tpz = symm_func.eta,symm_func.lambda,symm_func.zeta,symm_func.tpz
    #for individual species, aka row 1
    for atomindex in 1:n1
        for index2 in atomindex+1:n1
            for index3 in index2+1:n1
                g_vals=calc_one_symm_val(positions[atomindex],positions[index2],positions[index3],dist2_mat[atomindex,index2],dist2_mat[atomindex,index3],dist2_mat[index2,index3],f_mat[atomindex,index2],f_mat[atomindex,index3],f_mat[index2,index3],η,λ,ζ)

                g_vals .*= symm_func.tpz 
                g_vec[1,atomindex] += g_vals[1]
                g_vec[1,index2] += g_vals[2]
                g_vec[1,index3] += g_vals[3]
            end
        end
    end
    for atomindex in n1+1:N
        for index2 in atomindex+1:N
            for index3 in index2+1:N
                g_vals=calc_one_symm_val(positions[atomindex],positions[index2],positions[index3],dist2_mat[atomindex,index2],dist2_mat[atomindex,index3],dist2_mat[index2,index3],f_mat[atomindex,index2],f_mat[atomindex,index3],f_mat[index2,index3],η,λ,ζ)

                g_vals .*= symm_func.tpz 
                g_vec[1,atomindex] += g_vals[1]
                g_vec[1,index2] += g_vals[2]
                g_vec[1,index3] += g_vals[3]
            end
        end
    end
#this corresponds to 110 for copper and 100 for Zn
    for atomindex in 1:n1
        for index2 in atomindex+1:n1
            for index3 in n1+1:n2
                g_vals=calc_one_symm_val(positions[atomindex],positions[index2],positions[index3],dist2_mat[atomindex,index2],dist2_mat[atomindex,index3],dist2_mat[index2,index3],f_mat[atomindex,index2],f_mat[atomindex,index3],f_mat[index2,index3],η,λ,ζ)

                g_vals .*= symm_func.tpz 
                g_vec[2,atomindex] += g_vals[1]
                g_vec[2,index2] += g_vals[2]
                g_vec[3,index3] += g_vals[3]
            end
        end
    end
# this corresponds to 100 for copper and 110 for Zn
    for atomindex in 1:n1
        for index2 in n1+1:N
            for index3 in index2+1:N
                g_vals=calc_one_symm_val(positions[atomindex],positions[index2],positions[index3],dist2_mat[atomindex,index2],dist2_mat[atomindex,index3],dist2_mat[index2,index3],f_mat[atomindex,index2],f_mat[atomindex,index3],f_mat[index2,index3],η,λ,ζ)

                g_vals .*= symm_func.tpz 
                g_vec[3,atomindex] += g_vals[1]
                g_vec[2,index2] += g_vals[2]
                g_vec[2,index3] += g_vals[3]
            end
        end
    end


    return g_vec .+ symm_func.G_offset
end

calc_symm_vals! (generic function with 2 methods)

These would calculate the complete matrices, of length 2 or 3 (for radial vs angular) for a single symmetry value. That is, for our particular example, we go through each symmetry function and populate a combinatorically sound G matrix

In [26]:
function total_symm_calc(positions,dist2_mat,f_mat,radsymmfunctions,angsymmfunctions,nrad,nang,n1,n2)
    g_mat = zeros(nrad*2 + nang*3 , length(positions))

    for g_index in 1:nrad
        idx=(g_index-1)*2+1
        g_mat[idx:idx+1,:] = calc_symm_vals!(positions,dist2_mat,f_mat,g_mat[idx:idx+1,:],n1,n2,radsymmfunctions[g_index])
    end
    for g_index in 1:nang 
        idx = nrad*2 + (g_index-1)*3 + 1 

        g_mat[idx:idx+2,:] = calc_symm_vals!(positions,dist2_mat,f_mat,g_mat[idx:idx+2,:],n1,n2,angsymmfunctions[g_index])
    end
    return g_mat
end

total_symm_calc (generic function with 2 methods)

And then, we move on to calculation of the delta-matrix. In this case, we want to calculate a unique triple, retrieve the three individual delta-values and then inputting them into the correct vector in the same fashion as above. 

In [8]:
function new_radial_symm_val!(rnew_ij,r2_ij,fnew_ij,f2_ij,η,g_norm)
    return exponential_part(η,r_newij,fnew_ij*g_norm) - exponential_part(η,r2_ij,f2_ij*g_norm)
end
function calc_new_symmetry_value!(g_vector,atomindex,dist2_mat,new_dist2_vector,f_matrix,new_f_vector,n1,n2,symmetry_function::RadialType2)
    η,g_norm = symmetry_function.eta,symmetry_function.G_norm
    N = n1+n2
    #calculate 11
    for index2 in 1:n1
        if index2 != atomindex
            g_new = new_radial_symm_val!()

            g_vector[1,atomindex] = g_new
            g_vector[1,index2] = g_new
        end
    end
    #calculate 10
    for index2 in n1+1:N
        if index2 != atomindex
            g_new = new_radial_symm_val!()

            g_vector[2,atomindex] += g_new
            g_vector[2,index2] += g_new
        end
    end
    #invert if moved atom is type 0 
    if atomindex > n1
        g_vector = g_vector[2:-1:1,:]
    end

    return g_vector

end

calc_new_symmetry_value! (generic function with 1 method)

Lastly we have to look at calculating a new angular symmetry value. This should return three totally nonsymmetric values which are then assigned to their respective indices in the list. We will typically retrieve a matrix of three rows, the top row should be '111' the next '110' and the last '100'. If we calculate assuming automatically that our atom is of type 1, we calculate the double n1 loop and put it in the top row, the double n2 loop in the three row and the n1n2 loop in the middle. We can then invert the matrix entirely if we actually have atom of type 0.



In [9]:
function new_angular_symm_vals(newposition,position1,position2,position3,rnew_ij,rnew_ik,r2_ij,r2_ik,r2_jk,fnew_ij,fnew_ik,f_ij,f_ik,f_jk,η,λ,ζ,tpz)

    θ_new,θ_old = all_angular_measure(newposition,position2,position3,rnew_ij,rnew_ik,r2_jk),all_angular_measure(position1,position2,position3,r2_ij,r2_ik,r2_jk)

    θ_val_old,θ_val_new = theta_part(θ_old,λ,ζ),theta_part(θ_new,λ,ζ)

    exp_new,exp_old = exponential_part(η,rnew_ij,rnew_ik,r2_jk,fnew_ij,fnew_ik,f_jk),exponential_part(η,r2_ij,r2_ik,r2_jk,f_ij,f_ik,f_jk)

    return tpz*(θ_val_new.*exp_new .- θ_val_old.*exp_old)
end
function calc_new_symmetry_value!(g_vector,positions,newposition,atomindex,dist2_mat,new_dist2_vector,f_matrix,new_f_vector,n1,n2,symmetry_function::AngularType3)

    N = n1+n2 

    η,λ,ζ,tpz = symmetry_function.eta,symmetry_function.lambda,symmetry_function.zeta,symmetry_function.tpz
    #calculate 111
    for j_index in 1:n1
        if j_index != atomindex
            for k_index in j_index+1:n1
                if k_index != atomindex

                    g_vals = new_angular_symm_vals(newposition,positions[atomindex],positions[j_index],positions[k_index],new_dist2_vector[indexj],new_dist2_vector[indexk],dist2_mat[indexi,indexj],dist2_mat[indexi,indexk],dist2_mat[indexj,indexk],new_f_vector[indexj],new_f_vector[indexk],f_matrix[indexi,indexj],f_matrix[indexi,indexk],f_matrix[indexj,indexk],η,λ,ζ,tpz)

                    g_vector[1,atomindex] = g_vals[1]
                    g_vector[1,j_index] = g_vals[2]
                    g_vector[1,k_index] = g_vals[3]
                end
            end
        end
    end
    # calculate 100 for i and 110 for j,k
    for j_index in n1+1:N
        if j_index != atomindex
            for k_index in j_index+1:N
                if k_index != atomindex
                    g_vals = new_angular_symm_vals(newposition,positions[atomindex],positions[j_index],positions[k_index],new_dist2_vector[indexj],new_dist2_vector[indexk],dist2_mat[indexi,indexj],dist2_mat[indexi,indexk],dist2_mat[indexj,indexk],new_f_vector[indexj],new_f_vector[indexk],f_matrix[indexi,indexj],f_matrix[indexi,indexk],f_matrix[indexj,indexk],η,λ,ζ,tpz)

                    g_vector[3,atomindex] = g_vals[1]
                    g_vector[2,j_index] = g_vals[2]
                    g_vector[2,k_index] = g_vals[3]
                end
            end
        end
    end
    # The mixed term needs more careful consideration
    if atomindex > n1 
        #if Zn i and k are 110 j is 100, but first put row 2 into 3 to flip later
        g_vector[3,:] += g_vector[2,:]

        for j_index in 1:n1
            if j_index != atomindex
                for k_index in 1:n2
                    if k_index != atomindex
                        g_vals = new_angular_symm_vals(newposition,positions[atomindex],positions[j_index],positions[k_index],new_dist2_vector[indexj],new_dist2_vector[indexk],dist2_mat[indexi,indexj],dist2_mat[indexi,indexk],dist2_mat[indexj,indexk],new_f_vector[indexj],new_f_vector[indexk],f_matrix[indexi,indexj],f_matrix[indexi,indexk],f_matrix[indexj,indexk],η,λ,ζ,tpz)

                        g_vector[2,atomindex] = g_vals[1]
                        g_vector[3,j_index] = g_vals[2]
                        g_vector[2,k_index] = g_vals[3]
                    end
                end
            end
        end
    else
        #if Cu then i,j into 110 k into 100
        for j_index in 1:n1
            if j_index != atomindex
                for k_index in 1:n2
                    if k_index != atomindex
                        g_vals = new_angular_symm_vals(newposition,positions[atomindex],positions[j_index],positions[k_index],new_dist2_vector[indexj],new_dist2_vector[indexk],dist2_mat[indexi,indexj],dist2_mat[indexi,indexk],dist2_mat[indexj,indexk],new_f_vector[indexj],new_f_vector[indexk],f_matrix[indexi,indexj],f_matrix[indexi,indexk],f_matrix[indexj,indexk],η,λ,ζ,tpz)

                        g_vector[2,atomindex] = g_vals[1]
                        g_vector[2,j_index] = g_vals[2]
                        g_vector[3,k_index] = g_vals[3]
                    end
                end
            end
        end
    
    end
    # if atomindex > n1
    #     g_vector = g_vector[3:-1:1,:]
    # end

    return g_vector
end

calc_new_symmetry_value! (generic function with 2 methods)

In [11]:
function total_symm_calc(g_mat,positions,newposition,atomindex,dist2_mat,new_dist2_vector,f_mat,new_f_vector,radsymmfunctions,angsymmfunctions,nrad,nang,n1,n2)
    
    g_mat = zeros(nrad*2 + nang*3 , length(positions))

    for g_index in 1:nrad
        idx=(g_index-1)*2+1

        g_mat[idx:idx+1,:] = calc_new_symmetry_value!(g_mat[idx:idx+1,:],atomindex,dist2_mat,new_dist2_vector,f_matrix,new_f_vector,n1,n2,radsymmfunctions[g_index])
 
    end
    for g_index in 1:nang 
        idx = nrad*2 + (g_index-1)*3 + 1 

        g_mat[idx:idx+2,:] = calc_new_symmetry_value!(g_mat[idx:idx+2,:],positions,newposiiton,atomindex,dist2_mat,new_dist2_vector,f_matrix,new_f_vector,n1,n2,angsymmfunctions[g_index])
    end

    return g_mat
end

total_symm_calc (generic function with 1 method)

We have written the essential functions we aim to use. On to initialising states and doing some testing.

In [18]:
import ParallelTemperingMonteCarlo.EnergyEvaluation.set_variables

In [66]:
function set_variables(config::Config{N,BC,T},dist2_mat,pot::RuNNerPotential2Atom{nrad,nang}) where {N,BC,T} where {nrad,nang}
    f_matrix = cutoff_function.(sqrt.(dist2_mat),Ref(pot.r_cut))
    g_matrix = total_symm_calc(config.pos,dist2_mat,f_matrix,pot.radsymfunctions,pot.angsymfunctions,nrad,nang,30,8)
    return NNPVariables{T}(zeros(N) , zeros(N) , g_matrix , f_matrix ,zeros(nrad*2 + nang*3 , N) , zeros(N))
end

set_variables (generic function with 5 methods)

In [43]:
dis2mat=get_distance2_mat(start_config)
fmat = cutoff_function.(sqrt.(dis2mat),Ref(runnerpotential.r_cut))
#g_vec_test = zeros(3,38)
positions = start_config.pos
testsymmfunc = runnerpotential.angsymfunctions[1]

AngularType3{Float64}(0.0001, 1.0, 1.0, 11.338, 2, 0.0245373592605483, -0.00048489009301811386)

In [64]:
g_vec_test = zeros(3,38)
X = calc_symm_vals!(positions,dis2mat,fmat,g_vec_test,30,8,runnerpotential.angsymfunctions[1])


3×38 Matrix{Float64}:
  0.477141     0.217155      0.217155     …   0.0159347    0.0159347
 -0.00048489  -0.00048489   -0.00048489       0.0789258    0.0789258
  0.0109327    0.000335009   0.000335009     -0.00048489  -0.00048489

In [45]:
calc_one_symm_val(positions[1],positions[2],positions[3],dis2mat[1,2],dis2mat[1,3],dis2mat[2,3],fmat[1,2],fmat[1,3],fmat[2,3],testsymmfunc.eta,testsymmfunc.lambda,testsymmfunc.zeta)

3-element MVector{3, Float64} with indices SOneTo(3):
 0.1515414746623898
 0.26146459510776443
 0.26146459510776443

In [67]:
states,movestrat,results,nsteps,counter=initialisation(mc_params,temp,start_config,runnerpotential,ensemble)

MethodError: MethodError: no method matching initialise_energy(::Config{38, SphericalBC{Float64}, Float64}, ::Matrix{Float64}, ::NNPVariables{Float64}, ::NVTVariables{Float64}, ::RuNNerPotential2Atom{5, 26, 30, 8})

Closest candidates are:
  initialise_energy(::Any, ::Any, ::Any, ::Any, !Matched::EmbeddedAtomPotential)
   @ ParallelTemperingMonteCarlo ~/ParallelTemperingMonteCarlo.jl/src/EnergyEvaluation.jl:793
  initialise_energy(::Any, ::Any, ::Any, ::Any, !Matched::RuNNerPotential)
   @ ParallelTemperingMonteCarlo ~/ParallelTemperingMonteCarlo.jl/src/EnergyEvaluation.jl:798
  initialise_energy(::Any, ::Any, ::Any, ::NVTVariables, !Matched::AbstractDimerPotentialB)
   @ ParallelTemperingMonteCarlo ~/ParallelTemperingMonteCarlo.jl/src/EnergyEvaluation.jl:785
  ...
